## LSPR: Pygbe vs analytical (experimental data)

We run pygbe and the analytical solution for the Cext calculation for the cases we have experimental data. We compare the error between these runs and analyze the results. 

We run both silver and gold, using the data from [Johnson and Christy, 1972](https://journals.aps.org/prb/pdf/10.1103/PhysRevB.6.4370). We run meshes from 2K elements of a nanoparticle of radius 10nm.

### Silver



In [1]:
import numpy

In [2]:
#import data
lambda_s , n_s , k_s = numpy.loadtxt('gold_silver_water_raw-data/silver_JC72.txt', unpack=True)
lambda_s = lambda_s*1e3 #To convert to nm

### Water interpolation data for lambda_silver

We need to obtain the refractive index for the same wavelengths we have data for the silver case. We will save the data on a file for futere purposes. 

In [3]:
from data_analysis_helper import linear_interp

In [4]:
lambda_w , n_w , k_w = numpy.loadtxt('gold_silver_water_raw-data/water_HQ72.txt', unpack=True)
lambda_w = lambda_w*1e3 #To convert to nm

In [5]:
#linear interpolation
water_real_linear , water_imag_linear = linear_interp(lambda_w, n_w, k_w)

In [6]:
print(min(lambda_s))
print(min(lambda_w))
print(max(lambda_s))
print(max(lambda_w))

187.9
200.0
1937.0
2000.0


Our water data starts at a wavelength of 200 nm, therefore we can interpolate values smaller than that. So we will strip out the values for the silver wavelengths that are not in the water interpolation range. 

In [7]:
numpy.where(lambda_s<200.0)

(array([0, 1, 2, 3]),)

In [8]:
lambda_ws = lambda_s[4:]

In [9]:
n_s = n_s[4:]
k_s = k_s[4:]

In [10]:
#lambda_sw

In [11]:
n_ws = water_real_linear(lambda_ws)
k_ws = water_imag_linear(lambda_ws)

In [12]:
#numpy.savetxt('interp_data/water_refrac_nm.txt', list(zip(lambda_ws, n_ws, k_ws)))

In [13]:
refrac_silver = n_s + 1j * k_s
refrac_water  = n_ws + 1j * k_ws

In [14]:
refrac_silver

array([ 1.15 +1.296j,  1.18 +1.312j,  1.20 +1.325j,  1.22 +1.336j,
        1.25 +1.342j,  1.26 +1.344j,  1.28 +1.357j,  1.28 +1.367j,
        1.30 +1.378j,  1.31 +1.389j,  1.33 +1.393j,  1.35 +1.387j,
        1.38 +1.372j,  1.41 +1.331j,  1.41 +1.264j,  1.39 +1.161j,
        1.34 +0.964j,  1.13 +0.616j,  0.81 +0.392j,  0.17 +0.829j,
        0.14 +1.142j,  0.10 +1.419j,  0.07 +1.657j,  0.05 +1.864j,
        0.05 +2.07j ,  0.05 +2.275j,  0.04 +2.462j,  0.04 +2.657j,
        0.05 +2.869j,  0.05 +3.093j,  0.05 +3.324j,  0.06 +3.586j,
        0.05 +3.858j,  0.06 +4.152j,  0.05 +4.483j,  0.04 +4.838j,
        0.03 +5.242j,  0.04 +5.727j,  0.04 +6.312j,  0.04 +6.992j,
        0.04 +7.795j,  0.09 +8.828j,  0.13+10.1j  ,  0.15+11.85j ,
        0.24+14.08j ])

In [15]:
diel_silver = refrac_silver * refrac_silver 
diel_water  = refrac_water * refrac_water

In [16]:
l_diel_silver = [str(d) for d in diel_silver]
l_diel_water = [str(d) for d in diel_water]

In [17]:
for j in range(len(l_diel_silver)):
    l_diel_silver[j] = l_diel_silver[j][1:-1]

In [18]:
for j in range(len(l_diel_water)):
    l_diel_water[j] = l_diel_water[j][1:-1]

In [19]:
#diel_silv = list(diel_silver)
#diel_wat = list(diel_water)
lamb_ws = list(lambda_ws)

In [20]:
lst_d = list(zip(l_diel_water, l_diel_silver)) 

In [21]:
for j in range(len(lst_d)):
    lst_d[j] = list(lst_d[j])

In [22]:
#lst_d

In [23]:
wave_diel_sil_wat = list(zip(lamb_ws, lst_d))

In [24]:
wave_diel_sil_wat[23]

(381.5, ['1.7968866304+8.240198656e-09j', '-3.471996+0.1864j'])

In [25]:
import pygbe

In [26]:
from pygbe.util.read_data import readFields

In [27]:
from pygbe.main import main

In [28]:
field_test= readFields('sphere_complex.config')


In [29]:
field_test

{'E': ['1.7956+8.5e-9j', '-3.3829+0.19136j'],
 'LorY': ['1', '1'],
 'Nchild': ['1', '0'],
 'Nparent': ['0', '1'],
 'charges': ['0', '0'],
 'child': ['0'],
 'coulomb': ['0', '0'],
 'kappa': ['1e-12', '1e-12'],
 'parent': ['NA', '0'],
 'pot': ['0', '0'],
 'qfile': ['NA', 'NA']}

In [30]:
#results = main(['', '/home/ncclementi/Documents/git/pygbe_dev/pygbe/examples/lspr'], field=field_test, return_results_dict=True)


Run started on:
	Date: 2017/2/10
	Time: 18:26:16
	PyGBe version: 0.2.1+236.g6bf08a0
Config file: /home/ncclementi/Documents/git/pygbe_dev/pygbe/examples/lspr/sphere_complex.config
Parameter file: /home/ncclementi/Documents/git/pygbe_dev/pygbe/examples/lspr/sphere_complex.param
Geometry folder: /home/ncclementi/Documents/git/pygbe_dev/pygbe/examples/lspr/geometry
Running in: /home/ncclementi/Documents/git/pygbe_dev/pygbe/examples/lspr

Reading surface 0 from file geometry/sphere2K_R10
Time load mesh: 0.19775605201721191
Removed areas=0: 0

Total elements : 2048
Total equations: 4096
----------------------------

1 surfaces:

Surface 0:
	2048 elements
	Surface type       : dielectric_interface
	Cells              : 9
	Twigs              : 8
	Levels             : 2
	C0 size            : 9.978611958305848
	C0 box center      : 0.0, 0.0, 0.0
	Twig cell size     : 4.989305979152924
	Rbox/theta         : 9.978611958305848
	Analytic distance  : 2.197623822113438
	Elem. per sq Ang   : 1.6349357

In [ ]:
Cext = []
for wave, E in wave_diel_sil_wat:
    field_test['E'] = E #if here then 1st value of 'E' has to match 1st value in the E list 
    results = main(['', '/home/ncclementi/Documents/git/pygbe_dev/pygbe/examples/lspr'], field=field_test, lspr=(-1,wave), return_results_dict=True)
    Cext.append(results['Cext'])